In [1]:
# Imports and all that good stuff

import pandas as pd
import numpy as np
import matplotlib as plt
import requests
import json

from datetime import date
from bs4 import BeautifulSoup
%matplotlib inline
from matplotlib import style
style.use('seaborn-white')

In [2]:
toxic_data_url = "https://data.delaware.gov/resource/u2vy-dniy.csv"
pd.read_csv(toxic_data_url).columns.values # get column names

array(['accid', 'basiscode1', 'basiscode2', 'calc_off_dis',
       'calc_off_er_b', 'calc_off_rcy_b', 'calc_rel_b', 'cas_no',
       'cert_name', 'cert_title', 'chem_name', 'cont_e_address',
       'cont_name', 'cont_phone', 'fac_city', 'fac_county', 'fac_mailcity',
       'fac_mailcounty', 'fac_mailstate', 'fac_mailstreet1',
       'fac_mailstreet2', 'fac_mailzip', 'fac_name', 'fac_street1',
       'fac_zip', 'form_a', 'form_r', 'fugitive', 'fugitivebasis',
       'geocoded_facility_location', 'geocoded_facility_location_address',
       'geocoded_facility_location_city',
       'geocoded_facility_location_state',
       'geocoded_facility_location_zip', 'landfarm', 'landfarmbasis',
       'landfill', 'landfillbasis', 'landimp', 'landimp_othr',
       'landimp_othrbasis', 'landimpbasis', 'landothr', 'landothrbasis',
       'max_onsite', 'metal', 'metal_to_potw', 'naics', 'naics1', 'naics2',
       'naics3', 'naics4', 'naics5', 'naics6', 'non_potw_off_tre',
       'off_basis1', 'off_ba

In [39]:
columns = ['geocoded_facility_location','geocoded_facility_location_address','geocoded_facility_location_city','geocoded_facility_location_state','geocoded_facility_location_zip']
toxic_data = pd.read_csv(toxic_data_url, usecols=columns)

In [40]:
toxic_data.head() # To make sure we grabbed the right columns

,geocoded_facility_location,geocoded_facility_location_address,geocoded_facility_location_city,geocoded_facility_location_state,geocoded_facility_location_zip
0,NaN,1645 RIVER RD,DELAWARE CITY,DE,19706.0
1,NaN,436 CES/CC 600 CHEVRON AVE,DOVER AFB,DE,19902.0
2,NaN,436 CES/CC 600 CHEVRON AVE,DOVER AFB,DE,19902.0
3,NaN,436 CES/CC 600 CHEVRON AVE,DOVER AFB,DE,19902.0
4,NaN,436 CES/CC 600 CHEVRON AVE,DOVER AFB,DE,19902.0


In [43]:
toxic_data['geocoded_facility_location_zip'].isnull().values.any()

True

In [37]:
toxic_data['geocoded_facility_location_zip'].isnull()

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
       ...  
856    False
857    False
858    False
859    False
860    False
861    False
862    False
863    False
864    False
865    False
866    False
867    False
868    False
869    False
870    False
871    False
872    False
873    False
874    False
875    False
876    False
877    False
878    False
879    False
880    False
881    False
882    False
883    False
884    False
885    False
Name: geocoded_facility_location_zip, dtype: bool

Not all zipcodes in the `toxic_data` dataframe are the proper format, that is **#####**.

Going to define a function to cut off zipcode after the forst 5 numbers.

In [63]:
def zipcode(num):
    string_num = str(num)
    num = int(float(string_num[0:5]))
    return num

In [64]:
toxic_data['geocoded_facility_location_zip'].fillna(value=0,inplace=True) # I was getting errors with the zipcode functio
                                                                          # error raise about nan values, even though
                                                                          # I could not find those values. nan's replaced
                                                                          # by 0.

In [67]:
# apply zipcode function to zipcode column
toxic_data['geocoded_facility_location_zip'] = toxic_data['geocoded_facility_location_zip'].apply(zipcode)

In [68]:
toxic_data

,geocoded_facility_location,geocoded_facility_location_address,geocoded_facility_location_city,geocoded_facility_location_state,geocoded_facility_location_zip
0,NaN,1645 RIVER RD,DELAWARE CITY,DE,19706
1,NaN,436 CES/CC 600 CHEVRON AVE,DOVER AFB,DE,19902
2,NaN,436 CES/CC 600 CHEVRON AVE,DOVER AFB,DE,19902
3,NaN,436 CES/CC 600 CHEVRON AVE,DOVER AFB,DE,19902
4,NaN,436 CES/CC 600 CHEVRON AVE,DOVER AFB,DE,19902
5,NaN,376 HOLLY OAK LN,CHESWOLD,DE,19936
6,NaN,780 SCHOOLHOUSE RD,DELAWARE CITY,DE,19706
7,NaN,780 SCHOOLHOUSE RD,DELAWARE CITY,DE,19706
8,NaN,780 SCHOOLHOUSE RD,DELAWARE CITY,DE,19706
9,NaN,780 SCHOOLHOUSE RD,DELAWARE CITY,DE,19706


In [69]:
school_data_url = "https://data.delaware.gov/resource/tdwa-q9bt.json"

In [70]:
all_columns = pd.read_json(school_data_url).columns.values.tolist() # Get all column names as a list
wanted_columns = ['schoolname', 'schooltype','geocoded_location','street1','street2','state','zip'] 
school_data = pd.read_json(school_data_url)

In [71]:
[all_columns.remove(item) for item in wanted_columns] # remove wanted columns from all column list
                                                      # all column list can now be dropped from school_data using pd.drop()

[None, None, None, None, None, None, None]

In [72]:
school_data.head()

,city,county,districtcode,districtname,districttype,educationlevel,geocoded_location,hasec,haselementarygrade,hashighgrade,...,isungraded,lowestgrade,schoolcode,schoolname,schooltype,schoolyear,state,street1,street2,zip
0,Camden,Kent,10,Caesar Rodney School District,Regular,Elementary,"{'longitude': '-75.512466', 'human_address': '...",0,1,0,...,0,01,610,Allen Frear Elementary School,PUBLIC,2016,DE,238 Sorghum Mill Road,--,19934
1,Magnolia,Kent,10,Caesar Rodney School District,Regular,Kindergarten,"{'human_address': '{""address"":""Post Office Box...",0,0,0,...,0,KG,611,J. Ralph McIlvaine Early Childhood Center,PUBLIC,2016,DE,Post Office Box 258,11 East Walnut,19962
2,Dover,Kent,10,Caesar Rodney School District,Regular,Elementary,"{'human_address': '{""address"":""3100 Hawthorne ...",0,1,0,...,0,01,612,Major George S. Welch Elementary School,PUBLIC,2016,DE,3100 Hawthorne Drive,--,199015391
3,Camden,Kent,10,Caesar Rodney School District,Regular,Elementary,"{'longitude': '-75.55001', 'human_address': '{...",0,1,0,...,0,KG,615,Kent Elementary Intensive Learning Center,PUBLIC,2016,DE,5 Old North Road,--,199341247
4,Dover,Kent,10,Caesar Rodney School District,Regular,Elementary,"{'longitude': '-75.553832', 'human_address': '...",0,1,0,...,0,01,616,Nellie Hughes Stokes Elementary School,PUBLIC,2016,DE,3874 Upper King Road,--,19904


In [73]:
school_data.drop(all_columns, axis=1,inplace=True)

In [74]:
school_data.head() # ensure we get the columns we want

,geocoded_location,schoolname,schooltype,state,street1,street2,zip
0,"{'longitude': '-75.512466', 'human_address': '...",Allen Frear Elementary School,PUBLIC,DE,238 Sorghum Mill Road,--,19934
1,"{'human_address': '{""address"":""Post Office Box...",J. Ralph McIlvaine Early Childhood Center,PUBLIC,DE,Post Office Box 258,11 East Walnut,19962
2,"{'human_address': '{""address"":""3100 Hawthorne ...",Major George S. Welch Elementary School,PUBLIC,DE,3100 Hawthorne Drive,--,199015391
3,"{'longitude': '-75.55001', 'human_address': '{...",Kent Elementary Intensive Learning Center,PUBLIC,DE,5 Old North Road,--,199341247
4,"{'longitude': '-75.553832', 'human_address': '...",Nellie Hughes Stokes Elementary School,PUBLIC,DE,3874 Upper King Road,--,19904


Same problem with zipcodes. Will apply the `zipcode` function to the column **zip**.

In [75]:
school_data['zip'] = school_data['zip'].apply(zipcode)

In [76]:
school_data

,geocoded_location,schoolname,schooltype,state,street1,street2,zip
0,"{'longitude': '-75.512466', 'human_address': '...",Allen Frear Elementary School,PUBLIC,DE,238 Sorghum Mill Road,--,19934
1,"{'human_address': '{""address"":""Post Office Box...",J. Ralph McIlvaine Early Childhood Center,PUBLIC,DE,Post Office Box 258,11 East Walnut,19962
2,"{'human_address': '{""address"":""3100 Hawthorne ...",Major George S. Welch Elementary School,PUBLIC,DE,3100 Hawthorne Drive,--,19901
3,"{'longitude': '-75.55001', 'human_address': '{...",Kent Elementary Intensive Learning Center,PUBLIC,DE,5 Old North Road,--,19934
4,"{'longitude': '-75.553832', 'human_address': '...",Nellie Hughes Stokes Elementary School,PUBLIC,DE,3874 Upper King Road,--,19904
5,"{'longitude': '-75.533325', 'human_address': '...",Star Hill Elementary School,PUBLIC,DE,594 Voshells Mill/Star Hill Road,--,19901
6,"{'longitude': '-75.55001', 'human_address': '{...",W. B. Simpson Elementary School,PUBLIC,DE,5 Old North Road,--,19934
7,"{'longitude': '-75.512414', 'human_address': '...",F. Niel Postlethwait Middle School,PUBLIC,DE,2841 South State Street,--,19934
8,"{'longitude': '-75.538795', 'human_address': '...",W. Reily Brown Elementary School,PUBLIC,DE,360 Webbs Lane,--,19904
9,"{'longitude': '-75.552113', 'human_address': '...",Fred Fifer III Middle School,PUBLIC,DE,109 East Camden-Wyoming Avenue,--,19934
